# 課題（固有値・固有ベクトル）

- すべての課題を実行せよ
- すべての課題が完了したら Jupyter の File メニューから Download as -> HTML (.html) として結果をダウンロードし， Bb9 の「課題」からファイル添付で提出せよ

In [ ]:
# 初期化なので最初に実行してください
using Test
using LinearAlgebra

## 課題1: QR法

- 正方行列 A を QR 分解し，新たな行列 A=RQ を作成する
- この手続きを繰り返すと A は上三角行列に近づき，対角要素が行列Aの固有値となる

### qr2 の説明
- 引数
    - A: m行n列の行列
- 戻り値
    - Q: 互いに直交するn個の列ベクトルからなる行列．m行n列の行列
    - R: n行n列の上三角行列
- 関数内の変数
    - A[:,j]: 行列 $A$ の $j$ 列目のベクトル
    - Q[:,j]: 行列 $Q$ の $j$ 列目のベクトル
    - R[i,j]: 行列 $R$ の $i,j$ 要素
    - m: 行数
    - n: 列数

In [ ]:
function qr2(A)
    m,n = size(A)
    Q = copy(A)  # Aのコピーを作成
    R = zeros(n,n)
    R[1,1] = norm(Q[:,1])
    Q[:,1] /= R[1,1]
    for k = 2:n
        for i = 1:k-1
            R[i,k] = dot(Q[:,i], A[:,k])
            Q[:,k] -= R[i,k] * Q[:,i]
        end
        R[k,k] = norm(Q[:,k])
        Q[:,k] /= R[k,k]
    end
    return (Q,R)
end

### 課題1-1
- QR法によって固有値を求める関数 qralgorithm を作成せよ
- 関数が作成できたらテストを実行して **Test Passed** になることを確認せよ
- 関数がきちんとできていない場合 **Test Failed** が出力されるので **Test Passed**になるように backsolve を修正せよ

#### qralgorithmの説明
- 引数
    - A: 正方行列
- 戻り値
    - diag(A): 固有値のベクトル
- 関数内の変数
    - diag(A): 行列Aの対角要素のベクトル
    - Q: 行列AのQR分解から生成される直交行列
    - R: 行列AのQR分解から生成される上三角行列
    - t1, t2: 対角要素（固有値のベクトル）のノルム
    - err: 対角要素のノルムの相対誤差

In [ ]:
function qralgorithm(A)
    err = 1.0  # エラーを適当に入れておく
    t1 = diag(A) ## 対角要素ベクトル
    while err > 1.0e-10
        # コードを記述
        # (1) 行列AをQR分解
        # (2) R*Q として新しい行列A作成
        t2 = diag(A)
        err = maximum(abs.((t2 - t1) ./ t1))
        t1 = t2
    end
    return diag(A)
end

#### qralgorithmのテストプログラム
- 以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので関数を修正

In [ ]:
for k = 1:10
    s = rand(1:10)
    A = rand(s, s)
    A = A * A' # 固有値をすべて実数にするため
    eigenvalues = eigvals(A)
    println(@test  sort(qralgorithm(A)) ≈ sort(eigenvalues))
end

## 課題2: べき乗法

- $x_{k+1} = \frac{A x_{k}}{||A x_{k}||}$ を繰り返す
- $x_\infty$ は絶対値が最大となる固有値に対する固有ベクトル
- 固有ベクトル $x$ に対応する固有値はレイリー商で算出
$$
\lambda = \frac{v^T A v}{v^T v}
$$

### 課題2-1
- べき乗法によって固有値を求める関数 powermethod を作成せよ
- 関数が作成できたらテストを実行して **Test Passed** になることを確認せよ
- 関数がきちんとできていない場合 **Test Failed** が出力されるので **Test Passed**になるように修正せよ

### powermethodの説明
- 引数
    - A: 正方行列（2次元配列）
- 戻り値
    - lambda1: 最大固有値
    - x: 固有ベクトル
- 関数内の変数
    - lambda1, lambda2: 固有値
    - err: 固有値の相対誤差

In [ ]:
function powermethod(A)
    x = fill(1.0, size(A)[1])
    err = 1.0
    lambda1 = dot(x, A * x) / dot(x,x)
    while err > 1.0e-10
        # コードを記述
        # (1) Ax として x を更新
        # (2) x を norm(x) で正規化
        lambda2 = dot(x, A * x) / dot(x,x)
        err = abs(lambda2 - lambda1)/abs(lambda1)
        lambda1 = lambda2
    end
    (lambda1, x)
end

#### powermethodのテストプログラム
- 以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので関数を修正

In [ ]:
for k = 1:10
    s = rand(1:10)
    A = rand(s, s)
    A = A * A' # 固有値をすべて実数にするため
    lambda, x = powermethod(A)
    maxeigen = eigmax(A)
    println(@test  maxeigen ≈ lambda)
end

## 課題3: 逆べき乗法

- $x_{k+1} = \frac{A^{-1} x_{k}}{||A x_{k}||}$ を繰り返す
- $x_\infty$ は絶対値が最小となる固有値に対する固有ベクトル
- 固有ベクトル $x$ に対応する固有値はレイリー商で算出
$$
\lambda = \frac{v^T A v}{v^T v}
$$

### 課題3-1
- 後退代入によって上三角行列 $U$ と定数ベクトル $b$ からなる連立方程式を解く関数 backsolve を完成させよ
- 関数が作成できたらテストを実行して **Test Passed** になることを確認せよ
- 関数がきちんとできていない場合 **Test Failed** が出力されるので **Test Passed**になるように backsolve を修正せよ

#### backsolveの説明
- 引数
    - U: 上三角行列（2次元配列）
    - b: 定数ベクトル
- 戻り値
    - x: $Ux=b$を満たす解

In [ ]:
function backsolve(U, b)
    n = length(b)
    x = zeros(n) # x をゼロで初期化
    for i = n:-1:1 # i = n, n-1, ..., 1 の順番でforループ
        x[i] = b[i] 
        ### Uの非対角要素 U[i,j] に対する処理
        for j = n:-1:i+1 # j = n, n-1, ..., i+2, i+1 の順番でforループ
            ## ここに適切なコードを記述
        end
        ### Uの対角要素 U[i,i] に対する処理
            ## ここに適切なコードを記述
    end
    return x
end

#### backsolveのテストプログラム
- backsolve が作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているのでbacksolveを修正

In [ ]:
for k = 1:10
    s = rand(1:10)
    U = triu(rand(s,s))
    b = rand(s)
    println(@test backsolve(U, b) ≈ U \ b)
end

### 課題3-2
- 逆べき乗法によって固有値を求める関数 invpowermethod を作成せよ
- 関数が作成できたらテストを実行して **Test Passed** になることを確認せよ
- 関数がきちんとできていない場合 **Test Failed** が出力されるので **Test Passed**になるように修正せよ

### invpowermethodの説明
- 引数
    - A: 正方行列（2次元配列）
- 戻り値
    - lambda1: 最小固有値
    - x: 固有ベクトル
- 関数内の変数
    - Q, R: AのQR分解
    - lambda1, lambda2: 固有値
    - err: 固有値の相対誤差

In [ ]:
function invpowermethod(A)
    x = fill(1.0, size(A)[1])
    err = 1.0
    Q, R = qr2(A) # LU分解でも良い
    lambda1 = dot(x, A*x) / dot(x,x)
    while err > 1.0e-10
        # コードを記述
        # (1) Q, R, backsolve を使って A^{-1}x として x を更新
        # (2) x を norm(x) で正規化
        lambda2 =dot(x, A*x) / dot(x,x)
        err =  abs(lambda2 - lambda1)/abs(lambda1)
        lambda1 = lambda2
    end
    return (lambda1, x)
end


#### invpowermethodのテストプログラム
- 以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので関数を修正

In [ ]:
for k = 1:10
    s = rand(1:10)
    A = rand(s, s)
    A = A * A' # 固有値をすべて実数にするため
    lambda, x = invpowermethod(A)
    maxeigen = eigmin(A)
    println(@test  maxeigen ≈ lambda)
end